In [2]:
# Import the required libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from google.cloud import storage
from tensorflow.keras.callbacks import Callback

In [6]:
# Define dataset paths
train_dir = 'C:\\Users\\Sarim\\Documents\\Task 7\\dataset\\train'
test_dir = 'C:\\Users\\Sarim\\Documents\\Task 7\\dataset\\test'

# Define image dimensions and batch size
img_width, img_height = 260, 260
batch_size = 32

# Create data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'  # Use sparse categorical crossentropy
)

# Create a data generator without augmentation for testing
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'  # Use sparse categorical crossentropy
)

Found 17990 images belonging to 2 classes.
Found 4732 images belonging to 2 classes.


In [7]:
# Define the EfficientNetV2B2 model with random weights
base_model = tf.keras.applications.EfficientNetV2B2(
    include_top=False,
    weights=None,  # No pre-trained weights for random initialization
    input_shape=(img_width, img_height, 3),
    pooling=None,
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation='softmax')(x)  # Use 2 units for binary classification
x = Dropout(0.5)(x)

model = Model(inputs=base_model.input, outputs=x)

# Compile the model with sparse categorical crossentropy
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# Define custom callback to upload the model to Google Cloud Storage
class UploadModelCallback(Callback):
    def __init__(self, bucket_name, remote_dir, num_epochs=1):  # Change num_epochs to 1
        super().__init__()
        self.num_epochs = num_epochs
        self.bucket_name = bucket_name
        self.remote_dir = remote_dir
        self.local_dir = "/content/models"

        # Create a client to interact with Google Cloud Storage
        self.storage_client = storage.Client()

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) >= 1:  # Print the message after each epoch
            # Save the model locally
            filepath = os.path.join(self.local_dir, f"Cats_vs_Dogs_EffB0-{epoch + 1:02d}.h5")
            self.model.save(filepath, include_optimizer=True)

            # Upload the model to Google Cloud Storage
            remote_filepath = os.path.join(self.remote_dir, os.path.basename(filepath))
            bucket = self.storage_client.bucket(self.bucket_name)
            blob = bucket.blob(remote_filepath)
            blob.upload_from_filename(filepath)

            print(f"Model saved in {filepath} and uploaded to {remote_filepath}")

# Usage example:
bucket_name = "tibot-ml-labeling"
remote_dir = "models/cats_vs_dogs_task/sarim/EfficientNetV2B2_Random_Weight/model"
num_epochs = 10
upload_callback = UploadModelCallback(bucket_name, remote_dir, num_epochs)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [9]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client google-cloud-storage

                                              0.0/12.2 MB ? eta -:--:--
     -                                        0.5/12.2 MB 15.5 MB/s eta 0:00:01
     ---                                      1.0/12.2 MB 12.7 MB/s eta 0:00:01
     ---                                      1.2/12.2 MB 10.8 MB/s eta 0:00:02
     -----                                    1.8/12.2 MB 12.6 MB/s eta 0:00:01
     --------                                 2.7/12.2 MB 14.6 MB/s eta 0:00:01
     ------------                             3.8/12.2 MB 17.6 MB/s eta 0:00:01
     ----------------                         5.1/12.2 MB 20.3 MB/s eta 0:00:01
     -------------------                      6.0/12.2 MB 21.4 MB/s eta 0:00:01
     ------------------------                 7.3/12.2 MB 22.3 MB/s eta 0:00:01
     -----------------------------            9.1/12.2 MB 25.3 MB/s eta 0:00:01
     ----------------------------------      10.9/12.2 MB 29.7 MB/s eta 0:00:01
     --------------------------------------  12.